In [273]:
import json
import os
import random

import pandas as pd
from IPython.display import HTML, display

# Export data

In [274]:
train_essays = pd.read_csv("train_essays.csv")
train_essays = train_essays.query("prompt_id == 0")
train_essays.head()

,id,prompt_id,text,generated
0,0059830c,0,Cars. Cars have been around since they became ...,0
1,005db917,0,Transportation is a large necessity in most co...,0
2,008f63e3,0,"""America's love affair with it's vehicles seem...",0
3,00940276,0,How often do you ride in a car? Do you drive a...,0
4,00c39458,0,Cars are a wonderful thing. They are perhaps o...,0


In [275]:
with open("generated_essays_0.json") as jf:
    synt_texts = json.load(jf)
synt_texts_df = pd.DataFrame(synt_texts)
synt_texts_df.head()

,0
0,"global level, cars are a major contributor to..."
1,To inform fellow citizens about the advantage...
2,Smog. Traffic. Runaway emissions. Our love af...
3,Imagine a city where the streets are free of ...
4,Every day car usage has many negative conseque...


In [276]:
num_texts = len(synt_texts_df)
num_texts

50

In [277]:
# приводим датасеты в единый формат
synt_texts_df = synt_texts_df.rename(columns={synt_texts_df.columns[0]: "text"})
synt_texts_df["id"] = synt_texts_df.index
synt_texts_df = synt_texts_df.assign(prompt_id=0)
synt_texts_df = synt_texts_df.assign(generated=1)  # все тексты сгегерированы

In [40]:
i = random.randint(0, num_texts)
print(synt_texts_df.iloc[i].text)

Some  advantages for citizens in limiting car usage that can be drawn from the passage set include:

1) Reduced stress and improved quality of life. Residents of Vauban, Germany who chose to live in a car-free community report being "much happier" without cars. With less cars clogging the streets, citizens can walk, bike, and play more freely, reducing stress and creating a positive community atmosphere. 

2) Improved air quality and public health. Fewer cars means less vehicle emissions polluting the air, leading to reduced smog. For example, Bogota, Colombia has seen improvements in air quality after implementing a "Day Without Cars." And Paris had to issue driving bans due to dangerous smog levels caused in part by diesel emissions. Limiting cars can literally save lives and allow the public to breathe easier.  

3) Promotion of alternative sustainable transport. With cars off the streets, citizens are incentivized to use public transport, walk, or bike for their daily commutes. The

In [41]:
# убираем тег </essay>, стоящий в конце каждого эссе
synt_texts_df["text"] = synt_texts_df["text"].apply(
    lambda text: text.replace("</essay>", "")
)
print(synt_texts_df.iloc[i].text)

Some  advantages for citizens in limiting car usage that can be drawn from the passage set include:

1) Reduced stress and improved quality of life. Residents of Vauban, Germany who chose to live in a car-free community report being "much happier" without cars. With less cars clogging the streets, citizens can walk, bike, and play more freely, reducing stress and creating a positive community atmosphere. 

2) Improved air quality and public health. Fewer cars means less vehicle emissions polluting the air, leading to reduced smog. For example, Bogota, Colombia has seen improvements in air quality after implementing a "Day Without Cars." And Paris had to issue driving bans due to dangerous smog levels caused in part by diesel emissions. Limiting cars can literally save lives and allow the public to breathe easier.  

3) Promotion of alternative sustainable transport. With cars off the streets, citizens are incentivized to use public transport, walk, or bike for their daily commutes. The

In [42]:
synt_texts_df.head()

,text,id,prompt_id,generated
0,"global level, cars are a major contributor to...",0,0,1
1,To inform fellow citizens about the advantage...,1,0,1
2,Smog. Traffic. Runaway emissions. Our love af...,2,0,1
3,Imagine a city where the streets are free of ...,3,0,1
4,Every day car usage has many negative conseque...,4,0,1


In [43]:
# объединяем датасеты и перемешиваем порядок текстов
essays_for_labelling = pd.concat((synt_texts_df, train_essays.sample(num_texts)))
essays_for_labelling = essays_for_labelling.sample(frac=1)

In [44]:
# сохраняем датасет
essays_for_labelling.to_csv("essays_for_labelling.csv")

Так мы получили датасет, в котором 100 текстов: 50, написанных человеком, и 50, сгенерированных нейросетью

Разметка происходила в label studio. Подробности в README.

# Annotation analysis

## Loding annotation

In [307]:
annotated_df = pd.read_csv("annotation.csv")

In [308]:
annotated_df.head()

,Unnamed: 0,annotation_id,annotator,answer,created_at,generated,id,lead_time,other_features,prompt_id,value,text,updated_at
0,707,1,2,"[{""end"":125,""text"":""help help us us a world"",""...",2023-12-03T07:14:45.911309Z,0,829e720d,727.302,"{""text"":[""много ошибок"",""хромает логика"",""мног...",0,Human,Automobile accidents are very common in suburb...,2023-12-03T07:34:50.482105Z
1,2,30,2,"[{""end"":34,""text"":""Smog. Traffic. Runaway emis...",2023-12-03T16:54:07.663770Z,1,2,82.524,"{""text"":[""абзацы одинаковой длины"",""короткий т...",0,AI,Smog. Traffic. Runaway emissions. Our love af...,2023-12-03T16:54:07.663804Z
2,916,36,5,"[{""end"":124,""text"":""alll "",""start"":120,""labels...",2023-12-05T08:49:04.141663Z,0,a5639578,419.751,"{""text"":[""есть ошибки\/опечатки, у моделей каж...",0,Human,Have you ever walked outside and taken a deep ...,2023-12-05T08:49:04.141682Z
3,406,26,2,"[{""end"":3205,""text"":""See whats no cars does fo...",2023-12-03T16:24:03.000280Z,0,5098c1ea,122.998,"{""text"":[""обращение к читателю"",""парцелляция"",...",0,Human,Many people do not rely on their cars to take ...,2023-12-03T16:24:03.000305Z
4,26,38,5,"[{""end"":76,""text"":""\\n\""In German Suburb, Life...",2023-12-05T09:54:02.336822Z,1,26,155.104,"просто перечисление, что было в каждом тексте ...",0,AI,"""In German Suburb, Life Goes On Without Cars""...",2023-12-05T09:54:02.336858Z


In [309]:
# датасет был размечен не полностью, извлекаем размеченные тексты
annotated_df = annotated_df[annotated_df.value != ""]

In [310]:
print(f"Всего размечено {len(annotated_df)} текстов.")

Всего размечено 35 текстов.


## Mistakes analysis

In [311]:
annotated_df["value"] = annotated_df.value.apply(lambda v: 1 if v == "AI" else 0)

In [312]:
# соотношение лейблов из разметки
annotated_df.groupby("value").agg({"text": "count"})

,text
value,
0,18
1,17


In [313]:
# соотношение ground truth лейблов
annotated_df.groupby("generated").agg({"text": "count"})

,text
generated,
0,18
1,17


Соотношение лейблов в разметке и в исходных данных совпадает. Просто случайность или разметка прошла полностью без ошибок?

In [314]:
annotated_df["correct"] = annotated_df.generated == annotated_df.value
incorrect = annotated_df[~annotated_df["correct"]]

In [315]:
print(f"{len(incorrect)} текстов размечены неверно.")

6 текстов размечены неверно.


6 ошибок все же есть. Посмотрим на них поближе:

In [318]:
def display_text(text, spans):
    if spans and isinstance(spans, str):
        spans = json.loads(spans)
    else:
        spans = []
    style = "AI{background-color:#FF0000} Human{background-color:#008000}"
    colored_text = list(text)
    num_paragraphs = 0
    for s in spans:
        start = s["start"]
        end = s["end"]
        label = s["labels"][0]
        num_linebreaks = text[:start].count("\n")
        start = start + num_linebreaks
        end = end + num_linebreaks
        colored_text[start] = f"<{label}>{colored_text[start]}"
        colored_text[end] = f"{colored_text[end]}</{label}>"
    content = "".join(colored_text).replace("\n", "<br>")
    html = f"<style>{style}</style> <div> {content} </div>"
    display(HTML(html))

In [319]:
for index, row in incorrect.iterrows():
    print(f"Essay {row.id}")
    display_text(row.text, row.answer)
    print(f"Annotation: {row.value}")
    print(f"Ground truth: {row.generated}")
    print(f"Comments: {row.other_features}")
    print("\n-----\n")

Essay 3


Annotation: 0
Ground truth: 1
Comments: nan

-----

Essay 52906497


Annotation: 1
Ground truth: 0
Comments: может просто съехал markdown оформление?

-----

Essay 29


Annotation: 0
Ground truth: 1
Comments: кажется, что структура очень стандартная, как по шаблону из учебника

-----

Essay 19


Annotation: 0
Ground truth: 1
Comments: nan

-----

Essay edfefc3f


Annotation: 1
Ground truth: 0
Comments: nan

-----

Essay 62b480e1


Annotation: 1
Ground truth: 0
Comments: {"text":["слишком длинно","слишком правильный текст","повторяются The city of Bogota,Colombia, The romantic city of Paris"]}

-----



Видимо,на неправильные ответы натолкнули ошибки маркдауна (слипшиеся слова), частые повторения и шаблонная структура в человеческих текстах, ссылки на источники в сгенерированных текстах.

## Quality evalution

Сравним качество разметки у разных разметчиков.

In [295]:
annotated_df.groupby("annotator").agg({"text": "count", "correct": "mean"})

,text,correct
annotator,,
2,21,0.904762
3,4,0.500000
4,4,0.500000
5,6,0.833333


У разметчиков 3 и 4 качество получилось довльно низким, но, скорее всего, это связано с небольшим числом размеченных текстов. Во-первых, само количество нерепрентативно для оценки качества, во-вторых, на первых текстах они, скорее всего, "набивали руку". При этом на примере разметчиков 2 и 5 видно, что отличать сгенерированные тексты от написанных человеком все-таки возможно.

## Features of AI/Human Text

In [296]:
def collect_text_features(generated):
    collected_features = []
    features = annotated_df[annotated_df.generated == generated].other_features
    for f in features:
        try:
            f = json.loads(f)
            collected_features.extend(f.get("text", []))
        except:
            if isinstance(f, str):
                collected_features.append(f)
    return collected_features

На мой взгляд, идея с выделением фрагментов оказалась не очень удачной для последующего анализа, т.к. не всегда понятно, почему разметчик выделил тот или иной фрагмент и в целом это требует погружения в контекст кажого эссе. Кажется, что признаки, перечисляемые в свободной форме, лушче помогут составить общую картину.

In [297]:
synt_features = collect_text_features(generated=1)
human_features = collect_text_features(generated=0)

In [298]:
from collections import Counter

In [299]:
# признаки сгенерированных текстов
Counter(synt_features).most_common()

[('короткий текст', 4),
 ('скучный текст', 3),
 ('абзацы одинаковой длины', 2),
 ('просто перечисление, что было в каждом тексте из приложения к промпту; кажется это сильно выдает модель',
  1),
 ('нумерованный список', 1),
 ('источники явно не упомниаются', 1),
 ('короткое введение', 1),
 ('слишком правильный текст', 1),
 ('модель спалилась)', 1),
 ('кажется, что структура очень стандартная, как по шаблону из учебника', 1),
 ('слишком обычный текст...', 1),
 ('длинные перечисления', 1),
 ('четкая структура, как по пунктам', 1),
 ('нет заключения', 1)]

In [300]:
# признаки человеческих текстов
Counter(human_features).most_common()

[('обращение к читателю', 2),
 ('неформальный текст', 2),
 ('много ошибок', 1),
 ('хромает логика', 1),
 ('много конкретных примеров', 1),
 ('часто повторяются два слова подряд', 1),
 ('явные ссылки на источники', 1),
 ('есть ошибки/опечатки, у моделей кажется с этим все нормально должно быть',
  1),
 ('некоторые предложения какие-то косноязычные ', 1),
 ('парцелляция', 1),
 ('Похоже на человека по структуре, есть одно проблемное место с to 39yearlods',
  1),
 ('вопросительные предложения', 1),
 ('может просто съехал markdown оформление?', 1),
 ('много личных местоимений (I, our, us); автор говорит our country, AI себя явно ни к какой стране не причисляет)',
  1),
 ('есть ссылки на конкретные источники, возможно даже с указанием строк (вот эти 34, 24)',
  1),
 ('часто повторяется слово "great"', 1),
 ('цитаты', 1),
 ('упоминания цен', 1),
 ('однозначная позиция у автора', 1),
 ('слишком длинно', 1),
 ('слишком правильный текст', 1),
 ('повторяются The city of Bogota,Colombia, The roman

Некоторые из полученных признаков (например, наличие местоимений, списоков, повторений, длину текстов и абзацев) можно будет легко формализовать. Другие признаки (ошибки, скучность, "косноязыкость") формализовать будет сложнее, но так же возможно.

Таким образом, несмотря на то, что текстов было размечено не так много, нам удалось получить некоторые инсайты о различиях синтетических и человеческих текстов, которые можно будет использовать в будущем. Цель разметки в этом и состояла, поэтому можно сказать, что в каком-то виде она была достингута, хотя, скорее всего, нам понадобятся еще несколько итераций разметки.